## Convert a WoWSims export JSON to usable `.add` commands

dependencies:
1. Spell.dbc exported to SQL with stoneharry spell editor
2. python 3.11+
3. jupyter notebook, pandas, sqlalchemy

optional (`superdupermacro`) addon to paste in 1 macro https://felbite.com/addon/4135-superdupermacro/

In [1]:
from pathlib import Path
import json

In [2]:
data_folder = Path('wowsimdata')
json_path = data_folder / Path('t6_fury_warrior.json')

In [3]:
with open (json_path, 'r') as f:
    data = json.load(f)

In [4]:
items = data['player']['equipment']['items']

In [5]:
# Import spell data from `spell.dbc` that has been exported to DB
from sqlalchemy import create_engine
import pandas as pd

# Database connection settings
config = {
    'user': 'acore',
    'password': 'acore',
    'host': 'localhost',
    'port': 3306,
    'database': 'spelleditorcustomwrath'
}

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}')

# Execute the SELECT query
query = "SELECT `ID`, `Effect1`, `EffectMiscValue1`, `SpellName0`, `SpellRank0` FROM `spell`"
df_spell = pd.read_sql_query(query, engine)

# Print the DataFrame
print(df_spell)

# Pickle DataFrame
df_spell.to_pickle("./df_spell.pkl")

# Close the engine
engine.dispose()

# Database connection settings
config = {
    'user': 'acore',
    'password': 'acore',
    'host': 'localhost',
    'port': 3306,
    'database': 'acore_world'
}

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}')

# Execute the SELECT query
query = "SELECT `entry`, `spellid_1` FROM `item_template`"
df_item = pd.read_sql_query(query, engine)

# Print the DataFrame
print(df_item)

# Pickle DataFrame
df_item.to_pickle("./df_item.pkl")

# Close the engine
engine.dispose()

/tmp/ipykernel_501610/2563654969.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


          ID  Effect1  EffectMiscValue1                 SpellName0  \
0          1        5                 0       Word of Recall (OLD)   
1          3        5                 0  Word of Mass Recall (OLD)   
2          4        5                 0       Word of Recall Other   
3          5        1                 0         Stoneharry's Touch   
4          7        1                 0                    Suicide   
...      ...      ...               ...                        ...   
49851  90043        2                 0            Chain Lightning   
49852  90044        2                 0            Positive Charge   
49853  90045        2                 0            Negative Charge   
49854  90046        6                 0            Positive Charge   
49855  90047        6                 0            Negative Charge   

         SpellRank0  
0                    
1                    
2                    
3      Testing Rank  
4                    
...             ...  
49851

In [6]:
# find enchant spells from enchant effect
SPELL_EFFECT_ENCHANT_ITEM = 53
enchant_eff_to_id = {}
for item in items:
    enchant_id = item.get('enchant')
    if not enchant_id:
        continue
    df = df_spell.loc[(df_spell['Effect1'] == SPELL_EFFECT_ENCHANT_ITEM) & (df_spell['EffectMiscValue1'] == enchant_id)]
    if df.empty:
        print(f'ERROR: No match for `enchant_id` in spell.dbc {enchant_id}')
        continue
    spell_id = df['ID'].values[0]
    enchant_eff_to_id[spell_id] = enchant_eff_to_id.get(spell_id, 0) + 1
print(enchant_eff_to_id)



{35452: 1, 35439: 1, 34004: 1, 27960: 1, 27899: 1, 33995: 1, 35490: 1, 27954: 1, 27984: 2}


In [7]:
# print .add commands for enchantItems, gems, gear, glyphs
errors = []
for spell_id, amount in enchant_eff_to_id.items():
    df = df_item.loc[df_item['spellid_1'] == spell_id]
    if df.empty:
        errors.append(f'ERROR: No match for `spellid_1` in spell.dbc {spell_id}')
        continue
    entry = df['entry'].values[0]
    print(f'.add {entry} {amount}')

print('')

gems_total = {}
for i in items:
    if i.get('gems'):
        for j in i['gems']:
            if (j == 0): # sometimes gems are 0
                continue
            if j not in gems_total:
                gems_total[j] = 1
            else:
                gems_total[j] += 1
for gemId, amount in gems_total.items():
    print(f'.add {gemId} {amount}')

print('')

for i, item in enumerate(items):
    item_id = item.get('id')
    if item_id:
        print(f'.add {item_id}')

print('')

glyphs = data['player']['glyphs']
for _, itemId in glyphs.items():
    print(f'.add {itemId}')

print('\n'.join(errors))

.add 29192 1
.add 28910 1
.add 38940 1
.add 38913 1
.add 38897 1
.add 38933 1
.add 29535 1
.add 38910 1
.add 38925 2

.add 34220 1
.add 32193 12
.add 32226 2
.add 32217 1

.add 30972
.add 32260
.add 30979
.add 32323
.add 30975
.add 30861
.add 30969
.add 30032
.add 30977
.add 32345
.add 32335
.add 32497
.add 28830
.add 30627
.add 28439
.add 28438
.add 30724

.add 43432
.add 43418
.add 49084
.add 43397

